# Holiday-planning: a teaser on OR modelling and implementation
This notebook accompanies a demo on OR problem-solving ("How to plan your holidays using Operations Research?"), covering the following topics:
- understanding and refining the problem (in workshop)
- modelling: translating the problem into a mathematical model (in workshop)
- implementation: setting up the model using Python and PuLP, and using a solver to find the optimal solution (this notebook)
- result interpretation: understanding the output, visualizing the result and deriving insights (this notebook)

## Problem description in a nutshell
F likes to plan his holidays long in advance. Holidays are a scarce resource for him: he only has a limited yearly budget, and now he wants to make the most out of it.

Some of F’s considerations:
- He likes to have Fridays off
- He needs to do his share of covering school vacations (obligation)
- He likes to ski
- He really needs a 2-week vacation to go somewhere warm

- Being 2 days off? Not sufficient to rest
- The longer he is off, the better he rests
- More than 20 days in a row? Not even trying to get that approved

... oh and let’s be opportunistic: weekends and holidays come for free

In this notebook, we will be building a mathematical optimization model to help F plan his holidays, modelling some of the considerations above. We want
- to maximize the utility of taking days off
- to incentivize being off on specific days of the year, or on specific weekdays
- longer periods as those provide more rest
- at least one period with a minimum duration of 14 days

This notebook glues together the core steps and the building blocks of the optimization model. You can check the `src` folder for the `ModelConfig`  which defines the relevant parameters.


## Modelling approach
_Disclaimer: the notebooks implement a particular modelling approach. This is only one out of a range of possible modelling approach, and is considered the easiest one to grasp the combined aspects of day-level attributes and period-level attributes._

Instead of defining the decision variables representing "on which days to schedule a day off", a period-level modelling approach is used. A **period** is characterized by the starting date $d$ and the duration of the period in days $l$. We can then represent some examples for the month of May 2024 in the visual below. Red squares represent public holidays.
- (May 1, 2024; 5 days) in green require 2 days from the budget as May 1, 4 and 5 are either public holidays or weekends.
- (May 9, 2024; 4 days) in yellow requires 1 day from the budget
- (May 9, 2024; 5 days) in orange requires 2 days from the budget

For each of these periods, the cost in terms of holiday units and value / utility - as sum of marginal day-level value and duration-level value - can be predetermined.

![image](https://github.com/user-attachments/assets/8d963c3b-98cd-4d4e-93e0-d482a198aa85)

## Mathematical model formulation

### Variables and sets
$p_{d, l}$: binary variable to indicate whether we take time off starting on day $d$ for a duration of $l$ days (1) or not (0)

$P$: set of all periods (d, l) with start and end date within the planning period

$P_{\delta}  \subset P$: Subset of all existing periods which overlap with day $\delta$, i.e. meaning $\delta$ falls inbetween $d$ and $d+l-1$

### Parameters
$c_{d, l}$: number of units to be used from holiday budget for taking time off starting on day $d$ for a duration of $l$ days

$u_{d, l}$: utility / value we get from taking time off starting on day $d$ for a duration of $l$ days

$B$: holiday budget, e.g. we can take 30 days off on a yearly basis

### Objective function

$\max \sum_{p \in P}{u_{d,l} * p_{d, l}}$

### Constraint 1: holiday budget

$\sum_{p \in P}{c_{d,l} * p_{d,l}} \leq B$

### Constraint 2: single-day coverage
We can only get the value of a specific day once, and hence it does not make sense to have two periods which cover a specific date. To model this, we can add a constraint for each day $d$ in the year (or more generic: the planning period) and enforce that at most one period containing that date can be selected.

$\sum_{p \in P_{\delta}}{p_{d,l}} \leq 1$


## Data preprocessing

In [ ]:
import pulp
import os
from datetime import date, timedelta
import sys
import plotly.express as px
import importlib

sys.path.insert(0, "..")
from src.plotting.calendar import holiday_calendar_plot
from src.mdl.period import HolidayPeriod
import src.config as config
from src.utils.data_loader import CSVDataLoader
from src.utils.enums import DayType, Weekday
from src.utils.time import Time
import src.utils.calculators as calculators
import pandas as pd


importlib.reload(config)

### Optimization model configuration
The optimization model itself has quite a number of parameters that can be adjusted to reflect the preferences of the user or to control the model scope. Hence, we introduce a configuration class to store all of these parameters.

In [ ]:
# Generate all dates in the complete planning period
start_date = config.ModelConfig.PLANNING_PERIOD_START_DATE
end_date = config.ModelConfig.PLANNING_PERIOD_END_DATE

all_dates_in_period = [
    start_date + timedelta(days=i)
    for i in range(config.ModelConfig.NUMBER_DAYS_IN_YEAR + 1)
]
print(f"Planning holidays between {start_date} and {end_date}")

### Public holidays and weekends (aka "free days off")
As we do not have to take a day from our holiday budget for the public holidays, we can use those as "free days off".

In [ ]:
# Data import can be controlled by using the enum class
location = config.Location.BERLIN
year = config.ModelConfig.PLANNING_YEAR

data_loader = CSVDataLoader(os.path.join(".", "datasets"))
df_public_holidays = data_loader.load_public_holidays(year, location)

all_public_holidays = df_public_holidays["Date"].values

print(f"Public holidays in {location.value} for {year}: {len(all_public_holidays)}")
df_public_holidays

In [ ]:
all_weekend_days = Time.get_all_weekend_days(all_dates_in_period)
print(f"Number of weekend days: {len(all_weekend_days)}")

In [ ]:
os.makedirs("results", exist_ok=True)

baseline_dates_by_type = {DayType.PUBLIC_HOLIDAY: all_public_holidays, DayType.WEEKEND: all_weekend_days}

fig = holiday_calendar_plot(baseline_dates_by_type)
fig.update_layout(title=f"Overview of all public holidays and weekends in {year}")
fig.write_html(f"results/calendar_{location.value}_{year}_base.html")
fig.show()

### Create all possible holiday periods


In [ ]:
from src.utils.time import PeriodFactory

holiday_periods = PeriodFactory.generate_all_possible_holiday_periods(
    all_dates_in_period, config.ModelConfig.MAX_HOLIDAY_PERIOD_LENGTH
)

print(
    f"Number of all possible periods with up to {config.ModelConfig.MAX_HOLIDAY_PERIOD_LENGTH} days: {len(holiday_periods)}\n"
)
print("First set of periods:")
for period in holiday_periods[:5]:
    print(f"\t{period}")

### Derive cost for each holiday period
Remember that we do not have to take days off from our budget for those which are already set by the government, i.e. public holidays and weekends. Hence, the cost of a holiday period differs from the length of it.

In [ ]:
dict_cost_of_taking_day_off = (
    calculators.CalculateCost.generate_single_date_cost_lookup(
        all_dates_in_period, all_public_holidays
    )
)

print("Cost of taking day off on:")
for potential_date in all_dates_in_period[:5]:
    print(f"\t{potential_date}: {dict_cost_of_taking_day_off[potential_date]}")

In [ ]:
print("Cost of taking a subsequent period off")
for period in holiday_periods[:5]:
    print(
        f"\t{period}: {calculators.CalculateCost.for_holiday_period(period, dict_cost_of_taking_day_off)}"
    )

### Utility we gain from taking days off
We can also define a function that calculates the utility we gain from taking a specific period off. This can be used to model the preference for taking consecutive days off as well as for having specific days off. Hence, the total utility captures the following aspects:
1. marginal utility of being off on a single day
2. duration-based utility to incentivize taking consecutive days off

#### Duration dependent component

In [ ]:
# Besides the marginal value of being some time off, we also get additional value for being off for a longer period of time. This is represented by the following step function


possible_durations = range(0, 30)
length_based_utilities = [
    calculators.CalculateUtility.duration_dependent_component(i)
    for i in possible_durations
]
fig = px.line(
    x=possible_durations,
    y=length_based_utilities,
    title="Period-based value of taking consecutive days off",
)
fig.update_xaxes(title_text="Period durations (days)")
fig.update_yaxes(title_text="Utility")

#### (Marginal) utility for a single day off
This enables us to take care of specific preferences in taking days off

In [ ]:
from src.config import ModelConfig

print("Marginal value of taking day off on:")
preferred_weekdays = ModelConfig.PREFERRED_WEEKDAYS_OFF
preferred_dates_off = ModelConfig.PREFERRED_DATES_OFF

for date_to_check in all_dates_in_period[:5]:
    marginal_value = calculators.CalculateUtility.marginal_value_of_a_single_day(
        date_to_check, preferred_weekdays, preferred_dates_off
    )
    weekday = Weekday(date_to_check.isoweekday()).name
    print(f"\t{weekday.capitalize()[:3]} {date_to_check}: {marginal_value}")

#### Overall utility of a specific period

In [ ]:
print("Total value of taking a specific period off:")
for period in holiday_periods[:5]:
    utility = calculators.CalculateUtility.total_value_for_period(
        period, preferred_weekdays, preferred_dates_off
    )
    print(f"\t{period}: {utility}")

    period_based_component = calculators.CalculateUtility.duration_dependent_component(
        period.duration()
    )
    summed_marginal_component = (
        calculators.CalculateUtility.summed_marginal_value_for_period(
            period, preferred_weekdays, preferred_dates_off
        )
    )
    print(f"\t\t- duration-based component: {period_based_component}")
    print(f"\t\t- marginal component: {summed_marginal_component}")
    print()

In [ ]:
period_assessments = []
for period in holiday_periods:
    period_assessments.append({
        "start_date": period.start_date(),
        "end_date": period.end_date(),
        "nr_days": period.duration(),
        "cost": calculators.CalculateCost.for_holiday_period(period, dict_cost_of_taking_day_off),
        "utility": calculators.CalculateUtility.total_value_for_period(period, preferred_weekdays, preferred_dates_off),
    })
df_complete_overview = pd.DataFrame(period_assessments)
df_complete_overview.sort_values(by=["start_date", "end_date"], inplace=True)
df_complete_overview[df_complete_overview["utility"] > 0]

## Model implementation
Using these building blocks, we can now implement the optimization model itself.

In [ ]:
model = pulp.LpProblem("HolidayPlanning", pulp.LpMaximize)

### Decision variable creation
Create the variables $p_{d,l}$, one for each of the possible "options" we have: either a single line or a 2-line combination.
- Variables are binary (0 or 1) as we can select each option only once.
- We exclude the combinations that do not save buses as combining them would only introduces additional complexity

Other types of decision variables include integer ones (`pulp.LpInteger`) or continuous ones (`pulp.LpContinuous`).

In [ ]:
# Now we are ready to start creating the variables to decide whether we take a specific day off or not
dict_variables = {}
for period in holiday_periods:
    var_name = f"being_off_{period}"
    dict_variables[period] = pulp.LpVariable(cat=pulp.LpBinary, name=var_name)

### Objective function definition (a.k.a. "the goal")
The objective function is the mathematical representation of the goal we want to achieve. We will either minimize (in this particular example) or maximize towards this objective function.
$$
\max \sum_{(d, l) \in P}{u_{d,l} * x_{d, l}}
$$

In [ ]:
# Building the objective function which will aim to maximize the value of the time off
objective_function_elements = []
for period, decision_variable in dict_variables.items():
    value_of_period = calculators.CalculateUtility.total_value_for_period(period, preferred_weekdays, preferred_dates_off)
    
    new_element = value_of_period * decision_variable
    objective_function_elements.append(new_element)

objective_function = pulp.lpSum(objective_function_elements)
model.setObjective(objective_function)

### Constraints - budget
Constraints represent hard restrictions on the decisions we want to take: we have to make sure that these are fulfilled, otherwise the solution is not valid (infeasible).

Different types of constraints: 
- `LpConstraintEQ`: strict equality
- `LpConstraintGE`: greater-than or equal-to
- `LpConstraintLE`: less-than or equal-to

**Constraint 1**: we have only a limited number of days off we can take in a year. Hence, we need to make sure that we do not exceed the budget of days off.
$$
\sum_{p \in (d, l)}{c_{d,l} * x_{d,l}} \leq B
$$


In [ ]:
max_days_off = ModelConfig.HOLIDAY_BUDGET
lhs_elements = []

for period, decision_variable in dict_variables.items():
    cost = calculators.CalculateCost.for_holiday_period(period, dict_cost_of_taking_day_off)
    
    lhs_elements.append(decision_variable * cost)

budget_constraint = pulp.LpConstraint(
    e=pulp.lpSum(lhs_elements),
    sense=pulp.LpConstraintLE,
    rhs=max_days_off,
    name="BudgetConstraint",
)
model.addConstraint(budget_constraint)

### Constraints - avoid overlapping holiday periods
**Constraint 2**: we can get the utility of a specific day only once. 

In other words: holiday periods cannot be overlapping as this would mean we get the utility of a specific day twice. Especially if one day of the year would have a very high utility, we can expect an optimization solver to exploit this by taking two periods which overlap on this day.

To model this, we can add a constraint for each day $\delta$ in the year (or more generic: the planning period) and enforce that at most one period containing that date can be selected.

$$
\sum_{(d, l) \in P_{\delta}}{p_{d,l}} \leq 1 \quad \forall \delta \in 2025
$$


In [ ]:
dict_date_variable_coverage: dict[date, set[pulp.LpVariable]] = {datestamp: set() for datestamp in all_dates_in_period}

for period, decision_variable in dict_variables.items():
    for datestamp in period.all_days():
        dict_date_variable_coverage[datestamp].add(decision_variable)

dict_date_constraints: dict[date, pulp.LpConstraint] = {}
for datestamp, variables in dict_date_variable_coverage.items():
    type_c = pulp.LpConstraintLE

    constraint = pulp.LpConstraint(
        e=pulp.lpSum(variables),
        sense=pulp.LpConstraintLE,
        rhs=1,
        name=f"OnePeriodPerDay_{datestamp}",
    )
    dict_date_constraints[datestamp] = constraint
    model.addConstraint(constraint)

### Extension - must-have dates off
**Constraint 3**: we want to enforce that we are off on specific dates. For each enforced date $\delta$, at least one period covering that date must be selected.

$$
\sum_{(d, l) \in P_{\delta}}{p_{d,l}} \geq 1 \quad \forall \delta \in \text{EnforcedDates}
$$

In [ ]:
must_have_dates_off = ModelConfig.MUST_HAVE_DATES_OFF

for enforced_date in must_have_dates_off:
    variables_covering_date = dict_date_variable_coverage.get(enforced_date, set())
    
    if not variables_covering_date:
        print(f"Warning: No periods cover enforced date {enforced_date}")
        continue
    
    constraint = pulp.LpConstraint(
        e=pulp.lpSum(variables_covering_date),
        sense=pulp.LpConstraintGE,
        rhs=1,
        name=f"EnforcedDateOff_{enforced_date}",
    )
    model.addConstraint(constraint)

print(f"Added {len(must_have_dates_off)} enforced date constraints")

In [ ]:
_ = model.writeLP("holiday_planning.lp")

## Model solving and result interpretation
Once formulated, we can pass the model to a solver, which will take care of (trying to) finding the optimal solution. We will use CBC, which is an open-source alternative and comes with the PuLP package.

_Note: it is recommended to always set a time-limit. Some problems do not scale well and can take an indefinite time to solve._

**Solver statuses**
Solving the model can lead to different statuses:
- Infeasible: the model cannot be solve as there is not any possible solution satisfying all constraints
- No solution found: we ran out of time and did not find any feasible solution yet
- Unbounded: the objective function value can become infinite
- Optimal: problem could be solved to optimality ("we know for sure we have found the best possible solution")

In [ ]:
model.solve(pulp.PULP_CBC_CMD(timeLimit=20, msg=False))
if model.status != pulp.LpStatusOptimal:
    print("Model did not find an optimal solution")

In [ ]:
# Extract the solution by checking the holiday periods we should take off based on the model's optimal result
def get_selected_holiday_periods(
    dict_variables: dict[HolidayPeriod, pulp.LpVariable]
) -> list[HolidayPeriod]:
    selected_periods = []
    for period, decision_variable in dict_variables.items():
        if decision_variable.value() is None:
            continue

        if decision_variable.varValue > 0.99:
            selected_periods.append(period)

    return selected_periods

In [ ]:
selected_periods = get_selected_holiday_periods(dict_variables)

print(f"Selected {len(selected_periods)} holiday periods\n")
for period in selected_periods:
    cost = calculators.CalculateCost.for_holiday_period(
        period, dict_cost_of_taking_day_off
    )
    value = calculators.CalculateUtility.total_value_for_period(
        period, preferred_weekdays, preferred_dates_off
    )
    print(f"\t{period} -> cost: {cost} \t value: {round(value, 2)}")

In [ ]:
from copy import deepcopy


def get_all_days_off(all_periods: list[HolidayPeriod]) -> set[date]:
    all_dates_off = set()

    for period in all_periods:
        all_dates_off = all_dates_off.union(set(period.all_days()))

    return all_dates_off


In [ ]:

dates_by_type_1 = deepcopy(baseline_dates_by_type)
dates_by_type_1[DayType.DAY_OFF] = get_all_days_off(selected_periods)

fig = holiday_calendar_plot(dates_by_type_1)
fig.write_html(f"results/calendar_{location.value}_{year}_optimized.html")
fig.show()

### Extension - having at least period with a minimum duration
**Constraint 4**: we want to make sure that we have at least one period with a minimum duration of M days. This can be modelled by adding a constraint that enforces that at least one of the periods has a duration of at least M days.

Set $P^{\geq M}$ represents all periods $p_{d, l}: l \geq M$. Then we can add the following constraint:
$$
\sum_{(d, l) \in P^{\geq M}}{p_{d,l}} \geq 1
$$

In [ ]:
if config.ModelConfig.AT_LEAST_ONE_PERIOD_WITH_LENGTH is not None:
    periods_compliant_with_min_duration = []
    for period, decision_variable in dict_variables.items():
        if period.duration() < config.ModelConfig.AT_LEAST_ONE_PERIOD_WITH_LENGTH:
            continue
        periods_compliant_with_min_duration.append(decision_variable)

    single_min_duration_constraint = pulp.LpConstraint(
        e=pulp.lpSum(periods_compliant_with_min_duration),
        sense=pulp.LpConstraintGE,
        rhs=1,
        name="MinRestPeriod",
    )

    model.addConstraint(single_min_duration_constraint)

In [ ]:
model.solve(pulp.PULP_CBC_CMD(timeLimit=20, msg=False))
if model.status != pulp.LpStatusOptimal:
    print("Model did not find an optimal solution")

In [ ]:
selected_periods_2 = get_selected_holiday_periods(dict_variables)
all_dates_off_2 = get_all_days_off(selected_periods_2)

print(f"Selected {len(selected_periods)} holiday periods\n")
for period in selected_periods_2:
    cost = calculators.CalculateCost.for_holiday_period(period, dict_cost_of_taking_day_off)
    value = calculators.CalculateUtility.total_value_for_period(period, preferred_weekdays, preferred_dates_off)
    print(f"\t{period} -> cost: {cost} \t value: {round(value, 2)}")

In [ ]:
import src.plotting.calendar as calendar
importlib.reload(calendar)

dates_by_type_2 = deepcopy(baseline_dates_by_type)
dates_by_type_2[DayType.DAY_OFF] = all_dates_off_2


fig = holiday_calendar_plot(dates_by_type_2)
fig.update_layout(title=f"Recommended days off for your preferences in {year}")
fig.write_html(f"results/calendar_{location.value}_{year}_optimized_min_{config.ModelConfig.AT_LEAST_ONE_PERIOD_WITH_LENGTH}_days.html")
fig.show()